In [87]:
import numpy as np

In [88]:
X = np.array([[1,1,1], [1,0,1], [1,0,0], [1,0,0], [1,1,1], [0,1,1], [0,0,0], [1,0,1], [0,1,0], [1,0,0]])
y = np.array([1,1,0,0,1,0,0,1,1,0])

In [89]:
X.shape

(10, 3)

In [90]:
y.shape

(10,)

In [91]:
X[:5]

array([[1, 1, 1],
       [1, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [1, 1, 1]])

In [92]:
y[:5]

array([1, 1, 0, 0, 1])

In [93]:
def calculate_entropy(y):
    n = len(y)
    if n == 0:
        return 0
    p = np.sum((y == 1)) / n

    if p == 0 or p == 1:
        H = 0
    else:
        H = -(p * np.log2(p)) - ((1-p) * np.log2(1-p))
    return H

In [94]:
calculate_entropy(y)

1.0

In [95]:
def split_dataset(X, node_indices, feature):
    left_indices, right_indices = [], []
    for i in node_indices:
        if X[i, feature] == 1:
            left_indices.append(i)
        else:
            right_indices.append(i)
    return left_indices, right_indices

In [96]:
root_indices = [0,1,2,3,4,5,6,7,8,9]
left_indices, right_indices = split_dataset(X, root_indices, feature=0)
print("Left indices for feature 0: ", left_indices)
print("Right indices for feature 0: ", right_indices)

Left indices for feature 0:  [0, 1, 2, 3, 4, 7, 9]
Right indices for feature 0:  [5, 6, 8]


In [97]:
def calc_info_gain(X, y, node_indices, feature):
    left_indices, right_indices = split_dataset(X, node_indices, feature)
    X_node, y_node = X[node_indices], y[node_indices]
    X_left, y_left = X[left_indices], y[left_indices]
    X_right, y_right = X[right_indices], y[right_indices]
    H_node = calculate_entropy(y_node)
    H_left = calculate_entropy(y_left)
    H_right = calculate_entropy(y_right)
    n_node = len(node_indices)
    w_left = len(left_indices) / n_node
    w_right = len(right_indices) / n_node

    info_gain = H_node - ((w_left * H_left) + (w_right * H_right))
    return info_gain

In [98]:
calc_info_gain(X, y, root_indices, feature=0)

0.034851554559677034

In [99]:
calc_info_gain(X, y, root_indices, feature=1)

0.12451124978365313

In [100]:
calc_info_gain(X, y, root_indices, feature=2)

0.2780719051126377

In [101]:
def get_best_split(X, y, node_indices):
    num_features = X.shape[1]
    best_feature = -1
    max_info_gain = 0
    for feature in range(num_features):
        curr_info_gain = calc_info_gain(X, y, node_indices, feature)
        if curr_info_gain > max_info_gain:
            max_info_gain = curr_info_gain
            best_feature = feature
    return best_feature

In [102]:
get_best_split(X, y, root_indices)

2

In [107]:
tree = []
def build_tree_recursive(X, y, node_indices, branch_name, max_depth, curr_depth):
    if curr_depth == max_depth:
        formatting = " " * curr_depth + "-" * curr_depth
        print(f"{formatting} {branch_name} leaf nodes with indices {node_indices}")
        return
    
    best_feature = get_best_split(X, y, node_indices)
    tree.append((curr_depth, branch_name, best_feature, node_indices))

    formatting = '-' * curr_depth
    print(f"{formatting} Depth {curr_depth}, {branch_name}: Split on feature: {best_feature}")

    left_indices, right_indices = split_dataset(X, node_indices, best_feature)

    build_tree_recursive(X, y, left_indices, 'Left', max_depth, curr_depth+1)
    build_tree_recursive(X, y, right_indices, "Right", max_depth, curr_depth+1)


In [108]:
build_tree_recursive(X, y, root_indices, "Root", max_depth=2, curr_depth=0)

 Depth 0, Root: Split on feature: 2
- Depth 1, Left: Split on feature: 0
  -- Left leaf nodes with indices [0, 1, 4, 7]
  -- Right leaf nodes with indices [5]
- Depth 1, Right: Split on feature: 1
  -- Left leaf nodes with indices [8]
  -- Right leaf nodes with indices [2, 3, 6, 9]


In [109]:
tree

[(0, 'Root', 2, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 (1, 'Left', 0, [0, 1, 4, 5, 7]),
 (1, 'Right', 1, [2, 3, 6, 8, 9])]